# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,6.91,81,40,5.66,SJ,1691465680
1,1,port-aux-francais,-49.3500,70.2167,2.87,75,38,15.71,TF,1691465611
2,2,albany,42.6001,-73.9662,22.43,92,100,3.61,US,1691466089
3,3,atafu village,-8.5421,-172.5159,28.37,71,26,6.28,TK,1691466089
4,4,puerto ayora,-0.7393,-90.3518,24.19,88,100,2.68,EC,1691465708


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

SystemError: initialization of _internal failed without raising an exception

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
# My Criteria:
# A max temperature lower than 25 degrees but higher than 15
# Cloudiness less than 100
# Wind speed less than 10.0 m/s


weather_filter_df = city_data_df.loc[(city_data_df["Max Temp"] < 25)
                                       & (city_data_df["Max Temp"] > 15) 
                                       & (city_data_df["Cloudiness"] < 100)
                                       & (city_data_df["Wind Speed"] < 10.0)
                                      ]

# Drop any rows with null values
weather_filter_df.dropna()

# Display sample data
weather_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,bethel,41.3712,-73.4140,24.17,87,40,2.57,US,1691465342
13,13,tolanaro,-25.0319,46.9987,18.04,100,75,5.14,MG,1691465858
14,14,vila velha,-20.3297,-40.2925,21.97,92,0,3.60,BR,1691466091
24,24,adamstown,-25.0660,-130.1015,21.16,75,55,5.41,PN,1691465614
25,25,aykhal,66.0000,111.5000,22.42,34,71,4.96,RU,1691465666
...,...,...,...,...,...,...,...,...,...,...
538,538,kolokani,13.5728,-8.0339,23.21,90,54,3.09,ML,1691466186
541,541,tereze,43.9361,42.4434,21.62,73,0,1.07,RU,1691466186
543,543,kilembe,0.2000,30.0000,18.79,61,99,0.47,UG,1691466187
544,544,garden city,42.3256,-83.3310,19.36,88,75,3.09,US,1691466099


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filter_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,bethel,US,41.3712,-73.4140,87,
13,tolanaro,MG,-25.0319,46.9987,100,
14,vila velha,BR,-20.3297,-40.2925,92,
24,adamstown,PN,-25.0660,-130.1015,75,
25,aykhal,RU,66.0000,111.5000,34,
...,...,...,...,...,...,...
538,kolokani,ML,13.5728,-8.0339,90,
541,tereze,RU,43.9361,42.4434,73,
543,kilembe,UG,0.2000,30.0000,61,
544,garden city,US,42.3256,-83.3310,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
tolanaro - nearest hotel: Hôtel Mahavokey
vila velha - nearest hotel: Hotel Prainha
adamstown - nearest hotel: No hotel found
aykhal - nearest hotel: Уют
port lincoln - nearest hotel: Boston Hotel
avarua - nearest hotel: Paradise Inn
zhalantun - nearest hotel: 金百灵大酒店
ribeira grande - nearest hotel: Hospedaria JSF
kalevala - nearest hotel: Отель "ВелТ"
muisne - nearest hotel: Hostal Té de Menta
constantia - nearest hotel: Hotel Chérica
alice springs - nearest hotel: Aurora Alice Springs
brookings - nearest hotel: Quality Inn
hamilton - nearest hotel: North Vista Manor
shemonaikha - nearest hotel: Листон
levuka - nearest hotel: No hotel found
nogliki - nearest hotel: Ноглики
sovetskaya gavan' - nearest hotel: Гостинный Двор
fort bragg - nearest hotel: Airborne Inn Lodging
alofi - nearest hotel: Matavai Resort
yellowknife - nearest hotel: No hotel found
urengoy - nearest hotel: Геолог
chernomorskoye - nearest hotel: Море
ki

,City,Country,Lat,Lng,Humidity,Hotel Name
11,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
13,tolanaro,MG,-25.0319,46.9987,100,Hôtel Mahavokey
14,vila velha,BR,-20.3297,-40.2925,92,Hotel Prainha
24,adamstown,PN,-25.0660,-130.1015,75,No hotel found
25,aykhal,RU,66.0000,111.5000,34,Уют
...,...,...,...,...,...,...
538,kolokani,ML,13.5728,-8.0339,90,Hôtel kolokani
541,tereze,RU,43.9361,42.4434,73,No hotel found
543,kilembe,UG,0.2000,30.0000,61,Rwenzori Backpackers
544,garden city,US,42.3256,-83.3310,88,Quality Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map